# Projeto Módulo 1 
---

# Resumo

---

# Pontos Principais

---

## Pre processamento
---


## Importações de Bibliotecas

In [1]:
from matplotlib import pyplot as plt
import pandas as pd 
import seaborn as sns 

---

## O TabNet

Os dados utilizados neste projeto foram obtidos através da ferramenta TabNet do DataSUS, que esta disponível em: 

* https://datasus.saude.gov.br/informacoes-de-saude-tabnet/

<img src='./images/TabNet.png' width=90%>

Esta é a interface da ferramenta. Nela estão disponíveis vários dados, inclusive dados que não pertencem diretamente ao DataSUS, como dados Demograficos do IBGE. Os dados que vou utilizar são:

1. AIH aprovadas por Ano/Mês processamento segundo Unidade da Federação.

2. Taxa mortalidade por Ano/Mês processamento segundo Unidade da Federação

3. Valor total por Ano/Mês processamento segundo Unidade da Federação

4. Domicílios por Abastecimento de água, Coleta de lixo e Instalações Sanitárias segundo Unidade da Federação.

5. Media Moradores por domicilios segundo Unidade da Federação

6. Populacao residente segundo Unidade da Federação
    
    **Nota sobre itens 1, 2 e 3:** A partir do processamento de junho de 2012, houve mudança na classificação da natureza e esfera dos estabelecimentos. Com isso, temos que:
      
    1. Até maio de 2012 estas informações estão disponíveis como "Natureza" e "Esfera Administrativa".

    2. De junho de 2012 a outubro de 2015, estão disponíveis tanto como "Natureza" e "Esfera Administrativa", como "Natureza Jurídica" e "Esfera Jurídica".
      
    3. A partir de novembro de 2015, estão disponíveis como "Natureza Jurídica" e "Esfera Jurídica".
    

### Fontes 

* Ministério da Saúde - Sistema de Informações Hospitalares do SUS (SIH/SUS)

* IBGE - Censos Demográfico 2010

Estes dados podem ser obtidos nas seguintes caregorias sublinhadas em vermelho no TabNet: 

<img src='./images/Dados_disponiveis_tabnet.png' width=90%>

## Importações dos Dados

Vou importar os dados que estão dentro da pasta `./dados_datasus/`:

In [2]:
AIH_aprovadas = pd.read_csv('./dados_datasus/AIH aprovadas por AnoMês processamento segundo Unidade da Federacao.csv', encoding='ISO-8859-1', sep=';', engine='python', skiprows=4,skipfooter=12)
taxa_mortalidade = pd.read_csv('./dados_datasus/Taxa mortalidade por AnoMês processamento segundo Unidade da Federacao.csv', encoding='ISO-8859-1', sep=';', engine='python', decimal=',', skiprows=4,skipfooter=12)
valor_total = pd.read_csv('./dados_datasus/Valor total por AnoMês processamento segundo Unidade da Federacao.csv', encoding='ISO-8859-1', sep=';', engine='python', decimal=',', skiprows=4,skipfooter=12)
internacoes = pd.read_csv('./dados_datasus/Internacoes por AnoMês processamento segundo Unidade da Federacao.csv', encoding='ISO-8859-1', sep=';', engine='python', skiprows=3,skipfooter=12)
instalacoes_sanitarias_domicilio = pd.read_csv('./dados_datasus/Domicilios por Instalacoes sanitarias segundo Unidade da Federacao.csv', encoding='ISO-8859-1', sep=';', engine='python', skiprows=3,skipfooter=2)
abastecimento_agua_domicilio = pd.read_csv('./dados_datasus/Domicílios por Abastecimento de água segundo Unidade da Federacao.csv', encoding='ISO-8859-1', sep=';', engine='python', skiprows=3,skipfooter=2)
coleta_lixo_domicilio = pd.read_csv('./dados_datasus/Domicílios por Coleta de lixo segundo Unidade da Federacao.csv', encoding='ISO-8859-1', sep=';', engine='python', skiprows=3,skipfooter=2)
media_moradores_domicilio = pd.read_csv('./dados_datasus/Media MoradoresDomicilios segundo Unidade da Federacao.csv', encoding='ISO-8859-1', sep=';', engine='python',decimal=',', skiprows=3,skipfooter=8)
populacao_residente = pd.read_csv('./dados_datasus/Populacao residente segundo Unidade da Federacao.csv', encoding='ISO-8859-1', sep=';', engine='python', skiprows=3,skipfooter=17)

---
## Tratando os dados

Como o dataset `populacao_residente` do IBGE não tem os números nos índices das macrorregiões, vou adicionar esta series em todos os dataframes no lugar da coluna `Unidade da Federação`, isso vai ser feito criando um dicionário para automatizar a reatribuição das colunas.

Alguns desses dataframes tem séries temporais (`AIH_aprovadas`,`taxa_mortalidade`, `valor_total` e `internacoes`) onde uma pequena parte dos registros estão no tipo `object`, mas são numéricos, então vou transforma-los.

Outros dataframes que armazenam os dados de abastecimento de agua, instalações sanitarias e coleta de lixo são divididas em varios grupos. O tratamento que será feito é dividir cada dataframe em duas categorias: `Rede geral` e `Rede alternativa`.

Vou considerar uma `Rede Geral` os domicílios que tem abastecimento de água por uma rede de canalização, instalação sanitaria por meio de uma rede geral de esgoto ou pluvial e coleta de lixo feita por caçamba ou serviço de limpeza.

Já os domicílios que tem abastecimento de água por meio de poço, nascente, carro-pipa, cisterna, rio, açude, lago ou igarapé. Que usam instalações sanitarias como fossa séptica ou rudimendar, vala, rio, lago ou mar ou outro escoadouro, inclusive quem não tem instalação sanitária. E que descarta o lixo através da queima ou enterrando na propriedade, em terreno baldio ou logradouro, jogado em rio, lago, mar, ou outro destino, como a categoria `Rede Alternativa`.

In [3]:
Unidades_Federacao = populacao_residente['Unidade da Federação']

datasets = {'AIH_aprovadas': AIH_aprovadas,'taxa_mortalidade': taxa_mortalidade,'valor_total': valor_total,'internacoes': internacoes,'abastecimento_agua_domicilio' : abastecimento_agua_domicilio,'instalacoes_sanitarias_domicilio' : instalacoes_sanitarias_domicilio,'coleta_lixo_domicilio' : coleta_lixo_domicilio,'media_moradores_domicilio': media_moradores_domicilio,'populacao_residente': populacao_residente}

for dataset, conteudo in datasets.items():    
    #  Substituindo index
    conteudo['Unidade da Federação'] = Unidades_Federacao
    
    #  Substituindo registros com hífen
    datasets[dataset] = conteudo.replace('-', 'NaN')
    
    #  Referenciando novo conteudo.
    datasets[dataset] = conteudo.set_index('Unidade da Federação')
    
#  Reatribuindo referência as variáveis SIH
AIH_aprovadas = datasets['AIH_aprovadas'].replace('-','NaN').astype(float).fillna(method='ffill').astype(int)
taxa_mortalidade = datasets['taxa_mortalidade'].replace('-','NaN').replace('...','NaN').astype(float).fillna(method='ffill')
valor_total = datasets['valor_total'].replace('-','NaN').astype(float).fillna(method='ffill')
internacoes = datasets['internacoes'].replace('-','NaN').astype(float).fillna(method='ffill').astype(int)

#  Reatribuindo referência as variáveis
instalacoes_sanitarias_domicilio = datasets['instalacoes_sanitarias_domicilio'].replace('-','NaN').astype(float).fillna(0)
abastecimento_agua_domicilio = datasets['abastecimento_agua_domicilio'].replace('-','NaN').astype(float).fillna(0)
coleta_lixo_domicilio = datasets['coleta_lixo_domicilio'].replace('-','NaN').astype(float).fillna(0).astype(int)

#  Instalações sanitarias
instalacoes_sanitarias_domicilio['Rede Geral'] = instalacoes_sanitarias_domicilio[instalacoes_sanitarias_domicilio.columns[0]]
instalacoes_sanitarias_domicilio['Rede Alternativa'] = instalacoes_sanitarias_domicilio[instalacoes_sanitarias_domicilio.columns[1:7]].sum(axis=1)
instalacoes_sanitarias_domicilio = instalacoes_sanitarias_domicilio[['Rede Geral', 'Rede Alternativa']]

#  Abastecimento de agua em duas categorias
abastecimento_agua_domicilio['Rede Geral'] = abastecimento_agua_domicilio[abastecimento_agua_domicilio.columns[0]]
abastecimento_agua_domicilio['Rede Alternativa'] = abastecimento_agua_domicilio[abastecimento_agua_domicilio.columns[1:10]].sum(axis=1)
abastecimento_agua_domicilio = abastecimento_agua_domicilio[['Rede Geral', 'Rede Alternativa']]

#  Coleta de Lixo 
coleta_lixo_domicilio['Rede Geral'] = coleta_lixo_domicilio[coleta_lixo_domicilio.columns[:2]].sum(axis=1)
coleta_lixo_domicilio['Rede Alternativa'] = coleta_lixo_domicilio[coleta_lixo_domicilio.columns[2:-2]].sum(axis=1)
coleta_lixo_domicilio = coleta_lixo_domicilio[['Rede Geral','Rede Alternativa']]    

# Dados populacionais do IBGE
media_moradores_domicilio = datasets['media_moradores_domicilio'][['Média_moradores/domicílios']]
populacao_residente = datasets['populacao_residente'][['População_residente']]

datasets = {'AIH_aprovadas': AIH_aprovadas,'taxa_mortalidade': taxa_mortalidade,'valor_total': valor_total,'internacoes': internacoes,'abastecimento_agua_domicilio' : abastecimento_agua_domicilio,'instalacoes_sanitarias_domicilio' : instalacoes_sanitarias_domicilio,'coleta_lixo_domicilio' : coleta_lixo_domicilio,'media_moradores_domicilio': media_moradores_domicilio,'populacao_residente': populacao_residente}

for dataset, conteudo in datasets.items():    
#  Exibindo informações
    print(50*'-')
    print(5*'=', f'> Dataset: {dataset} <', 5*'=')
    print(50*'-')
    display(conteudo.sample(5))
    conteudo.info()
    print('\n')

--------------------------------------------------
===== > Dataset: AIH_aprovadas < =====
--------------------------------------------------


,2008/Jan,2008/Fev,2008/Mar,2008/Abr,2008/Mai,2008/Jun,2008/Jul,2008/Ago,2008/Set,2008/Out,...,2019/Dez,2020/Jan,2020/Fev,2020/Mar,2020/Abr,2020/Mai,2020/Jun,2020/Jul,2020/Ago,Total
Unidade da Federação,,,,,,,,,,,,,,,,,,,,,
Mato Grosso,8720,12469,12900,13487,13006,14734,13974,13937,12928,14148,...,13607,13342,13594,13187,11731,11945,11222,11346,10712,1990840
Bahia,33542,39976,47930,60957,65203,70367,63804,62940,61490,60821,...,50929,53421,52654,53703,47228,45766,45180,47227,44078,8573450
Rio Grande do Norte,8194,10169,11457,11641,12472,11585,11531,10838,10889,10354,...,10251,10604,10145,10328,9667,9214,9989,10477,9149,1552395
Rio de Janeiro,24432,29550,43093,41988,45288,47403,45349,46041,45598,43608,...,43619,46198,42644,42159,38095,44079,43471,46447,44434,6559937
Rondônia,3475,6650,4016,3988,4232,3630,6908,6987,6962,6202,...,8138,9516,7818,8027,6347,6491,5443,6761,5361,1124858


<class 'pandas.core.frame.DataFrame'>
Index: 27 entries, Rondônia to Distrito Federal
Columns: 153 entries, 2008/Jan to Total
dtypes: int64(153)
memory usage: 32.5+ KB


--------------------------------------------------
===== > Dataset: taxa_mortalidade < =====
--------------------------------------------------


,2008/Jan,2008/Fev,2008/Mar,2008/Abr,2008/Mai,2008/Jun,2008/Jul,2008/Ago,2008/Set,2008/Out,...,2019/Dez,2020/Jan,2020/Fev,2020/Mar,2020/Abr,2020/Mai,2020/Jun,2020/Jul,2020/Ago,Total
Unidade da Federação,,,,,,,,,,,,,,,,,,,,,
Minas Gerais,3.85,4.36,4.48,4.23,4.36,4.77,5.00,5.01,4.83,4.70,...,5.28,5.46,5.27,5.43,5.66,5.74,6.13,6.55,7.03,5.19
São Paulo,4.27,5.20,5.20,4.79,5.15,5.39,5.69,5.76,5.41,5.52,...,6.54,6.84,6.60,6.64,7.68,8.59,9.12,9.34,8.96,6.22
Tocantins,2.36,3.12,3.04,2.62,2.29,2.84,2.92,2.58,2.44,2.17,...,3.48,4.17,4.01,4.53,3.80,4.07,4.85,5.60,5.55,3.05
Santa Catarina,3.69,4.03,3.95,4.00,3.73,3.69,4.22,4.26,4.04,3.98,...,4.95,5.18,5.07,5.11,5.64,5.10,5.72,6.52,7.35,4.71
Rio de Janeiro,3.41,4.55,4.73,4.31,5.15,4.95,4.94,5.15,5.42,5.34,...,7.79,7.89,7.58,7.83,8.33,10.91,10.30,10.15,9.49,6.90


<class 'pandas.core.frame.DataFrame'>
Index: 27 entries, Rondônia to Distrito Federal
Columns: 153 entries, 2008/Jan to Total
dtypes: float64(153)
memory usage: 32.5+ KB


--------------------------------------------------
===== > Dataset: valor_total < =====
--------------------------------------------------


...

<class 'pandas.core.frame.DataFrame'>
Index: 27 entries, Rondônia to Distrito Federal
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   Rede Geral        27 non-null     int64
 1   Rede Alternativa  27 non-null     int64
dtypes: int64(2)
memory usage: 648.0+ bytes


--------------------------------------------------
===== > Dataset: media_moradores_domicilio < =====
--------------------------------------------------


,Média_moradores/domicílios
Unidade da Federação,
Bahia,3.2
Roraima,3.7
Mato Grosso,2.9
Amapá,4.0
Piauí,3.4


<class 'pandas.core.frame.DataFrame'>
Index: 27 entries, Rondônia to Distrito Federal
Data columns (total 1 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Média_moradores/domicílios  27 non-null     float64
dtypes: float64(1)
memory usage: 432.0+ bytes


--------------------------------------------------
===== > Dataset: populacao_residente < =====
--------------------------------------------------


,População_residente
Unidade da Federação,
Alagoas,3173421
Maranhão,6400012
Mato Grosso do Sul,2371865
Amapá,625650
Rio Grande do Norte,3153359


<class 'pandas.core.frame.DataFrame'>
Index: 27 entries, Rondônia to Distrito Federal
Data columns (total 1 columns):
 #   Column               Non-Null Count  Dtype
---  ------               --------------  -----
 0   População_residente  27 non-null     int64
dtypes: int64(1)
memory usage: 432.0+ bytes




# Análise dos Dados
---